In [1]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [2]:
menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "圖文選單 1",
  "chatBarText": "查看更多資訊",
  "areas": [
    {
      "bounds": {
        "x": 8,
        "y": 4,
        "width": 1234,
        "height": 831
      },
      "action": {
        "type": "message",
        "text": "我想更認識你"
      }
    },
    {
      "bounds": {
        "x": 8,
        "y": 852,
        "width": 1234,
        "height": 826
      },
      "action": {
        "type": "message",
        "text": "讓我看看你的點點滴滴"
      }
    },
    {
      "bounds": {
        "x": 1263,
        "y": 8,
        "width": 1229,
        "height": 827
      },
      "action": {
        "type": "message",
        "text": "校園經歷精彩嗎"
      }
    },
    {
      "bounds": {
        "x": 1263,
        "y": 856,
        "width": 1229,
        "height": 822
      },
      "action": {
        "type": "message",
        "text": "太神拉"
      }
    }
  ]
}
"""

In [3]:
import json
secretFileContentJson=json.load(open("./line_secret_key",'r'))

print(secretFileContentJson.get("channel_access_token"))
print(secretFileContentJson.get("secret_key"))
print(secretFileContentJson.get("self_user_id"))

BG+HZ2BI4laibJcTO2kie46l/LulEzfdwaBSQ4mvt81OkOrTA3gj5MUzl5wmkVrVtv6ptROH+JMPfdHHcTrGyOfAcmRukoYa4C/1zMf1Ce8+ejdS9bUwKfUESBS4aVb5DCo8N2+NquHvnkxCKlTwEQdB04t89/1O/w1cDnyilFU=
00335a496a54045ab220318d12e3ad8d
U526e4526e2a46e2f815b080d6fd4ac56


In [4]:
import requests

menuJson=json.loads(menuRawData)

createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'
createMenuRequestHeader={'Content-Type':'application/json','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

#print(createMenuRequestHeader)

lineCreateMenuResponse = requests.post(createMenuEndpoint,headers=createMenuRequestHeader,data=json.dumps(menuJson))

print(lineCreateMenuResponse)
print(lineCreateMenuResponse.text)

<Response [200]>
{"richMenuId":"richmenu-c38adb1ec6dcdc30486e0c3cb04869ab"}


In [5]:
# 取得菜單Id 
uploadRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")
print(uploadRichMenuId)
#'https://api.line.me/v2/bot/richmenu/{richMenuId}/content'

# 設定Line的遠端位置
uploadMenuEndpoint='https://api.line.me/v2/bot/richmenu/%s/content' % uploadRichMenuId
print(uploadMenuEndpoint)

# 設定消息的基本安全憑證
uploadMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 上傳照片
uploadImageFile=open("./images/richmenu.jpg",'rb')
lineUploadMenuResponse=requests.post(uploadMenuEndpoint,headers=uploadMenuRequestHeader,data=uploadImageFile)

print(lineUploadMenuResponse)
print(lineUploadMenuResponse.text)

richmenu-c38adb1ec6dcdc30486e0c3cb04869ab
https://api.line.me/v2/bot/richmenu/richmenu-c38adb1ec6dcdc30486e0c3cb04869ab/content
<Response [200]>
{}


In [6]:
# 取得菜單id
linkRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")

# 將菜單id與用戶id組合成遠端位置
linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (secretFileContentJson["self_user_id"], linkRichMenuId)
print(linkMenuEndpoint)

# 設定消息基本安全憑證
linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息
lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
print(lineLinkMenuResponse)
print(lineLinkMenuResponse.text)

https://api.line.me/v2/bot/user/U526e4526e2a46e2f815b080d6fd4ac56/richmenu/richmenu-c38adb1ec6dcdc30486e0c3cb04869ab
<Response [200]>
{}


In [7]:
# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUserMenuResponse=requests.get(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUserMenuResponse)
print(lineUserMenuResponse.text)

https://api.line.me/v2/bot/user/U526e4526e2a46e2f815b080d6fd4ac56/richmenu
<Response [200]>
{"richMenuId":"richmenu-c38adb1ec6dcdc30486e0c3cb04869ab"}


In [8]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="您好，我是小秘書，在這裡我能幫您介紹煜晉這位好青年，除此之外，我也可以回答您的小問題，當您需要我的幫助時，請盡量開口"),

]

In [9]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [10]:
#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "[::text:]尋找BD",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "開發者，尋求教學",
            "text": "[::text:]求助專家",
            "data": "Data2"
          }
        ],
)
)

In [11]:
template_message_dict = {
    "[::text:]more":buttons_template_message
}

In [12]:
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage
)
@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage("您好，輸入\"/help\"取得更多資訊")
    )

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Sep/2019 23:40:40] "POST / HTTP/1.1" 200 -
